## Генератор описания прогноза погоды

----

### Задания:

#### 1. Скачивание данных о состоянии погоды в городе с gismeteo.ru
Выберите город в России и найдите прогноз погоды в нем на ближайшие 10 дней на сайте gismeteo.ru.
Используя известные вам библиотеки для работы с протоколом http и html кодом, извлеките прогноз на ближайшие 10 дней.

Резльтатом сбора данных должна быть таблица со следующими строками:
* минимальная температура
* максимальна температура
* скорость ветра
* уровень осадков 

В столбцах таблицы должны быть даты и дни недели.

#### 2. Генерация описания прогноза

Прогноз погоды должен состоять из следующих (или подобным им) предложений, генерируемых по шаблонам (ниже три шаблона):
* В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2
    * *В четверг в НазваниеГорода потеплеет на 7 градусов по сравнению со средой*
* Скорость ветра изменится на X единиц в день1 по сравнению с день2.
    * *Скорость ветра изменится на 3 единицы в понедельник по сравнению с пятницей*
* Уровень осадков повысится / понизится на X единиц за Y дней. 
    * *Уровень осадков понится на 3.85 единиц за 7 дней*

Вместо НазваниеГорода подставьте название выбранного вами города, используя фунцкии для согласования существительных с предлогами. 

#### 3. Ответы на вопросы

---

### 1. Сбор данных

Скачаем информацию о погоде в городе Петрозаводск на 10 ближайших дней.

Информация доступна по ссылке - https://www.gismeteo.ru/weather-petrozavodsk-3934/10-days/

Подключим необходимые библиотеки:

In [64]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

Чтобы сайт не распознал в нас робота, передадим в header корректный User-Agent.

In [9]:
url = 'http://www.gismeteo.ru/weather-petrozavodsk-3934/10-days/'
headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'
      }
r = requests.get(url, headers = headers)

Сохраним страницу в локальный файл:

*Прим.: Полученный при первом скачивании файл прилагается вместе с данным notebook*

In [16]:
with open('data/test.html', 'w') as output_file:
  output_file.write(r.text)

Функция для открытия файла. Может быть использована при проверке для открытия той же страницы, что была получена при выполнении задания.

In [42]:
def read_file(filename):
    with open(filename) as input_file:
        text = input_file.read()
    return text

Для парсинга html будем использовать BeautifulSoup. Явно укажем парсер, чтобы избежать непредсказуемого поведения при запуске в другой системе или среде.

In [45]:
soup = BeautifulSoup(r.content, "lxml")
#soup = BeautifulSoup(read_file('data/test.html'), "lxml")

Распарсим блок виджета с датами и днями недели:

In [67]:
dates = []
days_list = soup.find('div',{'class':'widget__row'})
#print(days_list)
days = days_list.find_all('div',{'class':'w_date'})
for d in days:
    day = d.find('div',{'class':'w_date__day'}).text
    date = d.find('span',{'class':['w_date__date black','w_date__date weekend']}).text
    dates.append(day + ' ' + date.strip())
    
print(dates)

['Пн 19 фев', 'Вт 20', 'Ср 21', 'Чт 22', 'Пт 23', 'Сб 24', 'Вс 25', 'Пн 26', 'Вт 27', 'Ср 28']


Распарсим блок виджета с температурой:

In [68]:
temps_max = []
temps_min = []
temp_list = soup.find('div',{'class':'templine w_temperature'})
vals = temp_list.find_all('div',{'class':'value'})
for val in vals:
    mint = val.find('div',{'class':'mint'}).text
    maxt = val.find('div',{'class':'maxt'}).text
    temps_max.append(maxt)
    temps_min.append(mint)

print(temps_max)
print(temps_min)

['−8', '−7', '−8', '−12', '−12', '−14', '−11', '−8', '−8', '−3']
['−10', '−9', '−12', '−16', '−17', '−17', '−21', '−11', '−15', '−7']


Распарсим блок виджета с ветром:

In [60]:
winds = []
wind_list = soup.find('div',{'class':'widget__container'}).find_all('div',{'class':'widget__row widget__row_table'})[2]
ws = wind_list.find_all('div',{'class','w_wind'})
for w in ws:
    winds.append(w.text.strip())
    
print(winds)

['7', '10', '8', '8', '7', '6', '10', '12', '15', '14']


Распарсим блок виджета с осадками:

In [63]:
prec = []
prec_list = soup.find('div',{'class':'widget__container'}).find_all('div',{'class':'widget__row widget__row_table'})[3]
ps = prec_list.find_all('div',{'class','widget__item w_precipitation'})
for p in ps:
    prec.append(p.text.strip().replace(',','.'))
    
print(prec)

['1.85', '1', '0.4', '0', '0.3', '0.1', '0.4', '0.8', '1.6', '1.2']


Теперь можем собрать все в единую таблицу!

In [69]:
index = ['min_temp','max_temp','wind_speed','precipitation']
df = pd.DataFrame(data=[temps_min, temps_max, winds, prec],columns=dates,index=index)
df

,Пн 19 фев,Вт 20,Ср 21,Чт 22,Пт 23,Сб 24,Вс 25,Пн 26,Вт 27,Ср 28
min_temp,−10,−9,−12,−16,−17,−17,−21,−11,−15,−7
max_temp,−8,−7,−8,−12,−12,−14,−11,−8,−8,−3
wind_speed,7,10,8,8,7,6,10,12,15,14
precipitation,1.85,1,0.4,0,0.3,0.1,0.4,0.8,1.6,1.2


----
### 2. Генератор описания прогноза погоды 

Подключим необходимые библиотеки.

In [787]:
import pymorphy2
from pymorphy2.shapes import restore_capitalization #Необходим для сохранения регистра при склонении города

Создадим словарь, в котором ключем будет дата, а значением название колонки в таблице. Так как наши 10 дней попадают в период одного месяца, в качестве даты достаточно будет указать число (от 19 до 28). Так же создадим второй словарь, который бы ассоциировал сокращения дней недели с полным словом.

In [706]:
dates_d = {i+19:dates[i] for i in range(0,10)}
days_full = {'Пн':'Понедельник',
            'Вт':'Вторник',
            'Ср':'Среда',
            'Чт':'Четверг',
            'Пт':'Пятница',
            'Сб':'Суббота',
            'Вс':'Воскресенье'}

In [74]:
morph = pymorphy2.MorphAnalyzer()
city = 'Петрозаводск'

Прогноз погоды будет генерироваться на основе следующих шаблонов:

* *В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2*
* *Скорость ветра изменится на X единиц в день1 по сравнению с день2.*
* *Уровень осадков повысится / понизится на X единиц за Y дней.*

Для каждого шаблона напишем соответствующую функцию.

In [788]:
def temp_forecast(day_new, day_last):
    day_n = days_full[dates_d[day_new][:2]]
    day_l = days_full[dates_d[day_last][:2]]
    temp_n = round((int(df[dates_d[day_new]].iloc[0].replace('−','-')) + int(df[dates_d[day_new]].iloc[1].replace('−','-')))/2)
    temp_l = round((int(df[dates_d[day_last]].iloc[0].replace('−','-')) + int(df[dates_d[day_last]].iloc[1].replace('−','-')))/2)
    diff = temp_n - temp_l
    
    if temp_n > temp_l:
        change = 'потеплеет'
    if temp_n < temp_l:
        change = 'похолодает'
    if temp_n == temp_l:
        return 'Температура воздуха не изменится.'
    
    parsed_diff = morph.parse('единица')[0]
    parsed_degree = morph.parse('градусы')[0]
    prep_v = 'Во' if day_n == 'Вторник' else 'В'
    prep_s = 'со' if day_l == 'Вторник' or day_l =='Среда' else 'с'
    
    parsed_city = morph.parse(city)[0]
    parsed_day_n = morph.parse(day_n)[0]
    parsed_day_l = morph.parse(day_l)[0]
    
    s = ' '.join([prep_v, parsed_day_n.inflect({'accs'}).word, 'в', 
                  restore_capitalization(parsed_city.inflect({'loct'}).word, city), change,
                  'на', str(abs(diff)), parsed_degree.make_agree_with_number(abs(diff)).word,
                  'по сравнению', prep_s, parsed_day_l.inflect({'ablt'}).word, '.'])
    
    return s

In [790]:
def wind_forecast(day_new, day_last):
    day_n = days_full[dates_d[day_new][:2]]
    day_l = days_full[dates_d[day_last][:2]]
    diff = int(df[dates_d[day_new]].iloc[2])-int(df[dates_d[day_last]].iloc[2])
    
    parsed_diff = morph.parse('единица')[0]
    prep_v = 'во' if day_n == 'Вторник' else 'в'
    prep_s = 'со' if day_l == 'Вторник' or day_l =='Среда' else 'с'
    parsed_day_n = morph.parse(day_n)[0]
    parsed_day_l = morph.parse(day_l)[0]
    s = ' '.join(['Скорость ветра изменится на', str(diff),
                 parsed_diff.inflect({'accs'}).make_agree_with_number(diff).word,
                 prep_v, parsed_day_n.inflect({'accs'}).word, 'по сравнению', prep_s, parsed_day_l.inflect({'ablt'}).word, '.'])
    return s

In [699]:
def prec_forecast(n):
    diff = float(df.iloc[3,n-1])-float(df.iloc[3,0])
    parsed_day = morph.parse('день')[0]
    
    if diff > 0:
        change = 'повысится'
    if diff < 0:
        change = 'понизится'
    if diff == 0:
        return ' '.join(['Уровень осадков не изменится за', str(n),parsed_day.make_agree_with_number(n).word, '.'])
    
    diff = round(abs(diff),2)
    parsed_diff = morph.parse('единица')[0]

    
    s = ' '.join(['Уровень осадков', change, 'на', str(diff), parsed_diff.inflect({'accs'}).make_agree_with_number(diff).word,
                  'за', str(n), parsed_day.make_agree_with_number(n).word, '.'])
    return s

Проверим работу:

In [702]:
prec_forecast(2)

'Уровень осадков понизится на 0.85 единиц за 2 дня .'

In [791]:
wind_forecast(20,19)

'Скорость ветра изменится на 3 единицы во вторник по сравнению с понедельником .'

In [789]:
temp_forecast(20,19)

'Во вторник в Петрозаводске потеплеет на 1 градус по сравнению с понедельником .'

Выведем сразу все разом:

In [796]:
print(temp_forecast(21,19))
print(wind_forecast(21,19))
print(prec_forecast(3))

В среду в Петрозаводске похолодает на 1 градус по сравнению с понедельником .
Скорость ветра изменится на 1 единицу в среду по сравнению с понедельником .
Уровень осадков понизится на 1.45 единиц за 3 дня .


---
### 3. Ответы на вопросы

* *В каких других задачах (помимо описания прогноза погоды) может понадобиться генерировать текст по шаблонам? В каких задачах может понадобиться генерировать текст об изменении числовых показателей по шаблонам?*

**Ответ:**
Например, для генерации однотипных отчетов, отличающихся только некоторыми данными. Также можно использовать для автоматических ответов с почтовых ящиков или при написании простенького бота.

Генерация текста об изменении числовых показателей по шаблонам подойдет для ведения журнала показаний приборов.

* *Шаблоны, которые вы использовали в этом задании, имеют фиксированную структуру. Фактически, ваша задача заключалась в том, чтобы подставить в шаблон число и согласовать единицы измерения с этим числом или подставить в шаблон название города и согласовать его с предлогом. Как можно разнообразить эти шаблоны? Как знание синтаксической структуры предложения может помочь в этой задаче?*

**Ответ:**
Если собирать информацию о типах осадков - можно было бы включить их в шаблоны. Также менее строгие рамки шаблона привнесли бы больше разнообразия в результатах генерации, но при этом бы и усложнили задачу. 